v3 : adding additional metadata for survival prediction.

In [22]:
import time
run_start = time.time()

import pandas as pd
import json
import glob
import os
from collections import Counter
import numpy as np
import tarfile

TCGA-RC-A7SB-01A-21R-A352-07
│    │  │    │ │  │ │    │ │
│    │  │    │ │  │ │    │ └─ Analyte (07 = RNA)
│    │  │    │ │  │ │    └─── Plate ID
│    │  │    │ │  │ └──────── Center ID
│    │  │    │ │  └────────── Portion order
│    │  │    │ └───────────── Sample vial
│    │  │    └─────────────── Sample type (01 = Tumeur, 11 = Normal)
│    │  └──────────────────── Patient ID
│    └─────────────────────── Tissue Source Site
└───────────────────────────── Project (TCGA)

# Chargement des données transcriptomiques

In [23]:
base_dir = "Project_dataset_LIHC_liver_extrc/gdc_download_20251124_133032.676140"     # RNA seq data

# Trouver tous les fichiers .tsv
files = glob.glob(os.path.join(base_dir, "**/*.tsv"), recursive=True)

print(f"📂 {len(files)} fichiers RNA-Seq trouvés")

# Créer le dictionnaire dfs
dfs = {}

for f in files:
    # Extraire le file_id (nom du dossier parent)
    # file_id = f.split("/")[-2]        # path for linux/macos
    file_id = f.split("\\")[-2]         # path for windows
    
    # Charger le fichier TSV
    df = pd.read_csv(f, sep="\t", comment="#")
    
    # Stocker dans le dictionnaire
    dfs[file_id] = df

print(f"✅ {len(dfs)} DataFrames chargés dans dfs")
print(f"\n📋 Exemples de file_id :")
for i, file_id in enumerate(list(dfs.keys())[:3]):
    print(f"  {i+1}. {file_id}")

📂 405 fichiers RNA-Seq trouvés
✅ 405 DataFrames chargés dans dfs

📋 Exemples de file_id :
  1. 0036fcec-eaed-430b-9a23-5efb2d2cc7f2
  2. 003ad4b1-dc1b-49bc-8ce2-d6dbd1250fee
  3. 00fb4b52-e6a4-4ad9-bbed-584e25851aca


In [24]:
for file_id, df in dfs.items():
    df_clean = df.loc[~df["gene_id"].astype(str).str.startswith("N_")]
    df_clean = df_clean.reset_index(drop=True)
    dfs[file_id] = df_clean


In [25]:
dfs

{'0036fcec-eaed-430b-9a23-5efb2d2cc7f2':                   gene_id   gene_name       gene_type  unstranded  \
 0      ENSG00000000003.15      TSPAN6  protein_coding        4632   
 1       ENSG00000000005.6        TNMD  protein_coding           0   
 2      ENSG00000000419.13        DPM1  protein_coding         950   
 3      ENSG00000000457.14       SCYL3  protein_coding         298   
 4      ENSG00000000460.17    C1orf112  protein_coding         157   
 ...                   ...         ...             ...         ...   
 60655   ENSG00000288669.1  AC008763.4  protein_coding           0   
 60656   ENSG00000288670.1  AL592295.6          lncRNA         226   
 60657   ENSG00000288671.1  AC006486.3  protein_coding           0   
 60658   ENSG00000288674.1  AL391628.1  protein_coding           2   
 60659   ENSG00000288675.1  AP006621.6  protein_coding          11   
 
        stranded_first  stranded_second  tpm_unstranded  fpkm_unstranded  \
 0                2298             2334   

# Extraction des métadonnées

## Extraction de biospecimen

In [26]:
# Extraire l'archive
tar_path = 'Project_dataset_LIHC_liver_extrc/biospecimen.cart.2025-11-24.tar.gz'
extract_dir = 'Project_dataset_LIHC_liver_extrc/biospecimen_data'

with tarfile.open(tar_path, 'r:gz') as tar:
    tar.extractall(path=extract_dir)
    print(f"✅ Archive extraite dans : {extract_dir}")

# Lister les fichiers extraits
files = os.listdir(extract_dir)
print("\nFichiers extraits :")
for f in files:
    print(f"  - {f}")

# Maintenant charger sample.tsv
sample_info = pd.read_csv(
    os.path.join(extract_dir, 'sample.tsv'),
    sep='\t'
)

print("\n📊 Aperçu de sample.tsv :")
print(sample_info.head())

✅ Archive extraite dans : Project_dataset_LIHC_liver_extrc/biospecimen_data

Fichiers extraits :
  - aliquot.tsv
  - analyte.tsv
  - portion.tsv
  - sample.tsv
  - slide.tsv

📊 Aperçu de sample.tsv :
  project.project_id                         cases.case_id cases.submitter_id  \
0          TCGA-LIHC  109436aa-a655-429b-8d3b-1a43385c9016       TCGA-DD-AAW0   
1          TCGA-LIHC  109436aa-a655-429b-8d3b-1a43385c9016       TCGA-DD-AAW0   
2          TCGA-LIHC  109436aa-a655-429b-8d3b-1a43385c9016       TCGA-DD-AAW0   
3          TCGA-LIHC  834cff01-3c12-42fa-abbe-ae0ae168c568       TCGA-DD-AAVR   
4          TCGA-LIHC  834cff01-3c12-42fa-abbe-ae0ae168c568       TCGA-DD-AAVR   

  samples.biospecimen_anatomic_site samples.biospecimen_laterality  \
0                               '--                            '--   
1                               '--                            '--   
2                               '--                            '--   
3                               '

In [27]:
sample_info

,project.project_id,cases.case_id,cases.submitter_id,samples.biospecimen_anatomic_site,samples.biospecimen_laterality,samples.catalog_reference,samples.composition,samples.current_weight,samples.days_to_collection,samples.days_to_sample_procurement,...,samples.specimen_type,samples.state,samples.submitter_id,samples.time_between_clamping_and_freezing,samples.time_between_excision_and_freezing,samples.tissue_collection_type,samples.tissue_type,samples.tumor_code,samples.tumor_code_id,samples.tumor_descriptor
0,TCGA-LIHC,109436aa-a655-429b-8d3b-1a43385c9016,TCGA-DD-AAW0,'--,'--,'--,'--,'--,'--,0,...,Solid Tissue,released,TCGA-DD-AAW0-01Z,'--,'--,'--,Tumor,'--,'--,Primary
1,TCGA-LIHC,109436aa-a655-429b-8d3b-1a43385c9016,TCGA-DD-AAW0,'--,'--,'--,'--,'--,1737,'--,...,Peripheral Blood NOS,released,TCGA-DD-AAW0-10A,'--,'--,'--,Normal,'--,'--,Not Applicable
2,TCGA-LIHC,109436aa-a655-429b-8d3b-1a43385c9016,TCGA-DD-AAW0,'--,'--,'--,'--,'--,1737,'--,...,Unknown,released,TCGA-DD-AAW0-01A,'--,'--,'--,Tumor,'--,'--,Primary
3,TCGA-LIHC,834cff01-3c12-42fa-abbe-ae0ae168c568,TCGA-DD-AAVR,'--,'--,'--,'--,'--,2355,'--,...,Peripheral Blood NOS,released,TCGA-DD-AAVR-10A,'--,'--,'--,Normal,'--,'--,Not Applicable
4,TCGA-LIHC,834cff01-3c12-42fa-abbe-ae0ae168c568,TCGA-DD-AAVR,'--,'--,'--,'--,'--,'--,0,...,Solid Tissue,released,TCGA-DD-AAVR-01Z,'--,'--,'--,Tumor,'--,'--,Primary
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1092,TCGA-LIHC,7fb03840-5153-4dc6-a302-5a89aa4e1fb6,TCGA-ED-A8O6,'--,'--,'--,'--,'--,97,'--,...,Solid Tissue,released,TCGA-ED-A8O6-01A,'--,'--,'--,Tumor,'--,'--,Primary
1093,TCGA-LIHC,7fb03840-5153-4dc6-a302-5a89aa4e1fb6,TCGA-ED-A8O6,'--,'--,'--,'--,'--,97,'--,...,Peripheral Blood NOS,released,TCGA-ED-A8O6-10A,'--,'--,'--,Normal,'--,'--,Not Applicable
1094,TCGA-LIHC,08cbbb39-d05a-48b0-a966-5212c5923c8b,TCGA-RG-A7D4,'--,'--,'--,'--,'--,530,'--,...,Peripheral Blood NOS,released,TCGA-RG-A7D4-10A,'--,'--,'--,Normal,'--,'--,Not Applicable
1095,TCGA-LIHC,08cbbb39-d05a-48b0-a966-5212c5923c8b,TCGA-RG-A7D4,'--,'--,'--,'--,'--,530,'--,...,Solid Tissue,released,TCGA-RG-A7D4-01A,'--,'--,'--,Tumor,'--,'--,Primary


## Metadata.json

In [28]:
with open('Project_dataset_LIHC_liver_extrc/metadata.cart.2025-11-24.json', 'r') as f:
    metadata = json.load(f)

# Extraire entity_submitter_id
downloaded_samples = set()
for entry in metadata:
    if 'associated_entities' in entry and len(entry['associated_entities']) > 0:
        entity_id = entry['associated_entities'][0]['entity_submitter_id']
        downloaded_samples.add(entity_id)

# Vérification : combien d'échantillons contiennent vraiment des données transcriptomiques

In [29]:
# Extraire l'identifiant de l'échantillon depuis l'aliquot
# L'entity_submitter_id est de type "TCGA-RC-A7SB-01A-21R-A352-07"
# Mais samples.submitter_id est de type "TCGA-RC-A7SB-01A"
# Il faut extraire les premiers segments

# Extraire les 4 premiers segments (TCGA-RC-A7SB-01A)
downloaded_sample_prefixes = set()
for entity_id in downloaded_samples:
    # Format: TCGA-XX-XXXX-XXA-XXR-XXXX-XX
    # On garde: TCGA-XX-XXXX-XXA
    parts = entity_id.split('-')
    if len(parts) >= 4:
        sample_id = '-'.join(parts[:4])  # TCGA-RC-A7SB-01A
        downloaded_sample_prefixes.add(sample_id)

print(f"Identifiants d'échantillons extraits : {len(downloaded_sample_prefixes)}")

# Filtrer biospecimen pour correspondance

sample_info_filtered = sample_info[
    sample_info['samples.submitter_id'].isin(downloaded_sample_prefixes)
]

print(f"\n Échantillons qui ont du RNA-Seq : {len(sample_info_filtered)}")


if len(sample_info_filtered) ==  len(downloaded_sample_prefixes):
    print("\n✅ SUCCÈS ! Toutes les correspondances sont trouvées !")
else:
    print("\n⚠️ Aucune correspondance trouvée, vérifier le format des IDs")

Identifiants d'échantillons extraits : 405

 Échantillons qui ont du RNA-Seq : 405

✅ SUCCÈS ! Toutes les correspondances sont trouvées !


# Mapping 1

In [30]:
file_to_sample = {}
for entry in metadata:
    file_id = entry['file_id']  # d2fea0af-ffc7-...
    entity_id = entry['associated_entities'][0]['entity_submitter_id']  # TCGA-RC-A7SB-01A-21R-A352-07
    
    # Extraire les 4 premiers segments
    sample_id = '-'.join(entity_id.split('-')[:4])  # TCGA-RC-A7SB-01A
    
    file_to_sample[file_id] = sample_id

print(file_to_sample)

{'34580e9c-83f1-4252-9e71-5b6767d75699': 'TCGA-RC-A7SB-01A', 'e1ab994c-401a-4b92-ae85-59a421b89b52': 'TCGA-CC-A7IE-01A', 'ff12abd3-0f45-4063-afa7-fa5cad973159': 'TCGA-RC-A6M4-01A', '293f1d34-2368-4382-809a-3b04083329ca': 'TCGA-CC-A3MB-01A', '6e812349-35e7-44e7-ac7c-a4bc19d81fd0': 'TCGA-BC-A10Q-11A', '84f52c89-db34-4a63-9491-a11cebe8fcde': 'TCGA-HP-A5MZ-01A', 'cc0010d2-52db-4b15-a750-dd57d148520b': 'TCGA-G3-AAV5-01A', '922f342f-27af-4934-9760-79d7fa0f04ce': 'TCGA-G3-A3CH-01A', 'e820ebb7-91de-45f6-b03d-f97fca659059': 'TCGA-BC-A10X-01A', '51c45ad0-3a73-4561-9a2c-7a4c00de52c5': 'TCGA-BC-A10X-11A', '28c03921-6a64-4ce6-9627-8734812e4809': 'TCGA-BC-A112-01A', 'd38093cf-9e44-4328-950a-37b81cf775ca': 'TCGA-ZP-A9D1-01A', '4471dc96-3a94-410f-ba82-b3846aedea59': 'TCGA-DD-AAE7-01A', '37de7b75-c885-44c8-9227-6600d37438df': 'TCGA-DD-A39W-11A', '9e88b031-abcc-43fb-b49a-5debab1276b5': 'TCGA-DD-A39Z-01A', '79ab15c3-648d-4e4d-8b06-5bca7fb64683': 'TCGA-2Y-A9H0-01A', '5a9a239f-82d6-4875-970e-bc57558691b4':

In [31]:
len(file_to_sample)

405

2ème Mapping sample ID -> sample type

In [32]:
sample_type_map = sample_info.set_index('samples.submitter_id')['samples.sample_type'].to_dict()
sample_type_map

{'TCGA-DD-AAW0-01Z': 'Primary Tumor',
 'TCGA-DD-AAW0-10A': 'Blood Derived Normal',
 'TCGA-DD-AAW0-01A': 'Primary Tumor',
 'TCGA-DD-AAVR-10A': 'Blood Derived Normal',
 'TCGA-DD-AAVR-01Z': 'Primary Tumor',
 'TCGA-DD-AAVR-01A': 'Primary Tumor',
 'TCGA-FV-A496-01Z': 'Primary Tumor',
 'TCGA-FV-A496-11A': 'Solid Tissue Normal',
 'TCGA-FV-A496-10A': 'Blood Derived Normal',
 'TCGA-FV-A496-01A': 'Primary Tumor',
 'TCGA-DD-AACN-01Z': 'Primary Tumor',
 'TCGA-DD-AACN-10A': 'Blood Derived Normal',
 'TCGA-DD-AACN-01A': 'Primary Tumor',
 'TCGA-G3-A7M5-01Z': 'Primary Tumor',
 'TCGA-G3-A7M5-01A': 'Primary Tumor',
 'TCGA-G3-A7M5-10A': 'Blood Derived Normal',
 'TCGA-UB-AA0U-01Z': 'Primary Tumor',
 'TCGA-UB-AA0U-10A': 'Blood Derived Normal',
 'TCGA-UB-AA0U-01A': 'Primary Tumor',
 'TCGA-DD-AAVQ-10A': 'Blood Derived Normal',
 'TCGA-DD-AAVQ-01A': 'Primary Tumor',
 'TCGA-DD-AAVQ-01Z': 'Primary Tumor',
 'TCGA-5R-AA1D-01A': 'Primary Tumor',
 'TCGA-5R-AA1D-01Z': 'Primary Tumor',
 'TCGA-5R-AA1D-10A': 'Blood Deriv

In [33]:
#vérification du mapping

for file_id, df in dfs.items():
    # CHECK
    # print(file_id)

    # Étape 1 : file_id → sample_id
    sample_id = file_to_sample[file_id]
    
    # Étape 2 : sample_id → sample_type
    sample_type = sample_type_map.get(sample_id, 'Unknown')
    
    print(f"Fichier {file_id} → Échantillon {sample_id} → Type: {sample_type}")

Fichier 0036fcec-eaed-430b-9a23-5efb2d2cc7f2 → Échantillon TCGA-G3-A25X-01A → Type: Primary Tumor
Fichier 003ad4b1-dc1b-49bc-8ce2-d6dbd1250fee → Échantillon TCGA-DD-AADS-01A → Type: Primary Tumor
Fichier 00fb4b52-e6a4-4ad9-bbed-584e25851aca → Échantillon TCGA-G3-AAV3-01A → Type: Primary Tumor
Fichier 01e802a3-1ae8-4601-9540-d99dba62728e → Échantillon TCGA-T1-A6J8-01A → Type: Primary Tumor
Fichier 023b8a0a-1b32-4feb-8b9d-0844dc0e7c6c → Échantillon TCGA-2Y-A9GU-01A → Type: Primary Tumor
Fichier 02957f01-c615-4576-8d45-473204cb284a → Échantillon TCGA-2Y-A9HB-01A → Type: Primary Tumor
Fichier 03e3e614-324f-4471-89fc-6b134605f364 → Échantillon TCGA-DD-A73E-01A → Type: Primary Tumor
Fichier 0444bb3d-0844-4333-97b0-e71faca95abb → Échantillon TCGA-EP-A3RK-11A → Type: Solid Tissue Normal
Fichier 048103cc-f5d1-4ad4-8b82-ea76715bfa53 → Échantillon TCGA-UB-A7MB-01A → Type: Primary Tumor
Fichier 071a3c5b-9378-4cc0-ae6a-b772630822d0 → Échantillon TCGA-DD-A73A-01A → Type: Primary Tumor
Fichier 074a7d

# Matrice d'expression

In [34]:
expression_data = {}

for file_id, df in dfs.items():

    sample_id = file_to_sample[file_id]
    sample_type = sample_type_map.get(sample_id, 'Unknown')
    
    # Extraire l'expression des gènes
    gene_expression = df.set_index('gene_name')['tpm_unstranded'] #tpm : normalisés
    
    # Étape 4 : Stocker dans le dictionnaire
    expression_data[sample_id] = gene_expression
    
    # (Optionnel) Afficher la progression
    if len(expression_data) % 50 == 0:
        print(f"✅ {len(expression_data)} fichiers traités...")

# Créer la matrice (transpose : lignes = échantillons, colonnes = gènes)
expression_matrix = pd.DataFrame(expression_data).T

print(f"\n✅ Matrice créée : {expression_matrix.shape}")
print(f"   → {expression_matrix.shape[0]} échantillons")
print(f"   → {expression_matrix.shape[1]} gènes")

# Ajouter les métadonnées
expression_matrix['sample_type'] = expression_matrix.index.map(sample_type_map)

# Créer les labels
expression_matrix['cancer_label'] = expression_matrix['sample_type'].apply(
    lambda x: 1 if pd.notna(x) and ('Tumor' in str(x) or 'Primary' in str(x)) else 0
)

# ajout du patient 
# Ajout du patient (numérotation 1, 2, 3, ...)
patient_id_to_number = {patient_id: i+1 for i, patient_id in enumerate(expression_matrix.index.str[:12].unique())}

expression_matrix['patient'] = expression_matrix.index.str[:12].map(patient_id_to_number) 

print("\n📊 Distribution des types :")
print(expression_matrix['sample_type'].value_counts())

print("\n📊 Distribution binaire (Tumeur=1 / Normal=0) :")
print(expression_matrix['cancer_label'].value_counts())

✅ 50 fichiers traités...
✅ 100 fichiers traités...
✅ 150 fichiers traités...
✅ 200 fichiers traités...
✅ 250 fichiers traités...
✅ 300 fichiers traités...
✅ 350 fichiers traités...
✅ 400 fichiers traités...

✅ Matrice créée : (405, 60660)
   → 405 échantillons
   → 60660 gènes

📊 Distribution des types :
sample_type
Primary Tumor          353
Solid Tissue Normal     49
Recurrent Tumor          3
Name: count, dtype: int64

📊 Distribution binaire (Tumeur=1 / Normal=0) :
cancer_label
1    356
0     49
Name: count, dtype: int64


# Ajout des données cliniques à la matrice

In [35]:
# Charger les fichier clinical.tsv 
clinical_data = pd.read_csv(
    'Project_dataset_LIHC_liver_extrc/clinical.cart.2025-11-24/clinical.tsv',
    sep='\t'
)

# Sélectionner les colonnes d'intérêt
clinical_subset = clinical_data[['cases.submitter_id', 
                                 'demographic.age_at_index', 'demographic.days_to_death', 'demographic.vital_status', 'demographic.gender', 
                                 'diagnoses.ajcc_pathologic_stage', 'diagnoses.tumor_grade',
                                 ]].copy()

# Supprimer les duplicatas (garder la première occurrence)
clinical_subset = clinical_subset.drop_duplicates(subset='cases.submitter_id')

# Créer un mapping patient_id -> age_at_index et days_to_death
age_map = clinical_subset.set_index('cases.submitter_id')['demographic.age_at_index'].to_dict()                 # âge lors du diagnostic
days_to_death_map = clinical_subset.set_index('cases.submitter_id')['demographic.days_to_death'].to_dict()      # jours de survie
vital_status_map = clinical_subset.set_index('cases.submitter_id')['demographic.vital_status'].to_dict()        # vivant ou mort
sex_map = clinical_subset.set_index('cases.submitter_id')['demographic.gender'].to_dict()                       # sexe
stage_map = clinical_subset.set_index('cases.submitter_id')['diagnoses.ajcc_pathologic_stage'].to_dict()        # stage de cancer
grade_map = clinical_subset.set_index('cases.submitter_id')['diagnoses.tumor_grade'].to_dict()                  # grade de cancer

# Extraire le patient ID (premiers 12 caractères) depuis l'index de expression_matrix
expression_matrix['patient_id'] = expression_matrix.index.str[:12]

# Mapper les valeurs
expression_matrix['age_at_index'] = expression_matrix['patient_id'].map(age_map)
expression_matrix['days_to_death'] = expression_matrix['patient_id'].map(days_to_death_map)
expression_matrix['vital_status'] = expression_matrix['patient_id'].map(vital_status_map)
expression_matrix['sex'] = expression_matrix['patient_id'].map(sex_map)
expression_matrix['stage'] = expression_matrix['patient_id'].map(stage_map)
expression_matrix['grade'] = expression_matrix['patient_id'].map(grade_map)

# Supprimer la colonne temporaire patient_id
expression_matrix = expression_matrix.drop(columns=['patient_id'])

print("\n✅ Données cliniques ajoutées :")
print(f"   → age_at_index : {expression_matrix['age_at_index'].notna().sum()} valeurs")
print(f"   → days_to_death : {expression_matrix['days_to_death'].notna().sum()} valeurs")
print(f"   → vital_status : {expression_matrix['vital_status'].notna().sum()} valeurs")
print(f"   → sex : {expression_matrix['sex'].notna().sum()} valeurs")
print(f"   → stage : {expression_matrix['stage'].notna().sum()} valeurs")
print(f"   → grade : {expression_matrix['grade'].notna().sum()} valeurs")


✅ Données cliniques ajoutées :
   → age_at_index : 405 valeurs
   → days_to_death : 405 valeurs
   → vital_status : 405 valeurs
   → sex : 405 valeurs
   → stage : 405 valeurs
   → grade : 405 valeurs


# Formatage des colonnes metadonnées et labels

La cancer_label est formatté juste au-dessus, nous formatons maintenant le survival_label

In [36]:
label_cols = ['patient', 'age_at_index', 'sample_type', 'vital_status', 'cancer_label', 'days_to_death']

for col in label_cols :
    if col != "patient" :
        print(f"{col} :")
        print(f"    Unique values : {expression_matrix[col].unique()} \n")

age_at_index :
    Unique values : ['73' '63' '58' '68' '55' '66' '24' '71' '59' '50' '38' '62' '56' '42'
 '70' '44' '76' '46' '52' '69' '75' '20' '37' '18' '74' '51' '48' '65'
 '45' '60' '67' '80' '72' '36' '78' '16' '43' '77' '57' '47' '53' '29'
 '64' '49' '26' '54' '81' '61' '35' '85' '23' '40' '31' '83' '39' '79'
 "'--" '17' '82' '84' '32' '33'] 

sample_type :
    Unique values : ['Primary Tumor' 'Solid Tissue Normal' 'Recurrent Tumor'] 

vital_status :
    Unique values : ['Alive' 'Dead'] 

cancer_label :
    Unique values : [1 0] 

days_to_death :
    Unique values : ["'--" '56' '1005' '1149' '97' '837' '581' '67' '612' '300' '1852' '366'
 '1490' '129' '359' '365' '2131' '171' '153' '315' '3125' '1694' '643'
 '194' '1560' '14' '827' '1624' '1372' '296' '195' '115' '627' '349' '931'
 '693' '103' '3258' '247' '770' '16' '1622' '101' '899' '1229' '596'
 '1791' '9' '432' '2456' '91' '214' '223' '633' '34' '1135' '415' '303'
 '660' '31' '27' '262' '649' '65' '272' '535' '344' '768' '

In [37]:
# Check if all dead individuals have survival data (days_to_death)
count = 0

for ind, vital_stat, days_death in zip(expression_matrix.index, expression_matrix["vital_status"], expression_matrix["days_to_death"]) :
    if vital_stat == "Dead" and days_death == "'--" :
        count += 1
        print(f"{ind} is dead but doesn't have survival data.")

print(f"{count} samples are dead but have no survival data (days_to_death).")

0 samples are dead but have no survival data (days_to_death).


In [38]:
# Convert days_to_death to numeric, handling string values
expression_matrix['days_to_death'] = pd.to_numeric(expression_matrix['days_to_death'],
                                                   errors="coerce")

# Convert days to years
expression_matrix['years_to_death'] = expression_matrix['days_to_death'] / 365

# Create survival_label: 0 if < 2 years, 1 if >= 2 years, NaN if missing
expression_matrix['survival_label'] = expression_matrix['years_to_death'].apply(
    lambda x: 0 if x < 2 else 1 if x >= 2 else np.nan
)
expression_matrix['survival_label'] = expression_matrix['survival_label'].astype('Int64')

print("\nSurvival label created.")
print(f"Survival label distribution:")
print(expression_matrix['survival_label'].value_counts(dropna=False))


Survival label created.
Survival label distribution:
survival_label
<NA>    245
0       107
1        53
Name: count, dtype: Int64


# Sauvegarde

In [39]:
labels = ['cancer_label', "survival_label"]
meta_cols = ['years_to_death', 'days_to_death', 'sample_type', 'patient', 'age_at_index', 'vital_status', 'sex', 'stage', 'grade']
gene_cols = [col for col in expression_matrix if col not in [labels, meta_cols]]

In [40]:
expression_matrix.columns[-10:]

Index(['cancer_label', 'patient', 'age_at_index', 'days_to_death',
       'vital_status', 'sex', 'stage', 'grade', 'years_to_death',
       'survival_label'],
      dtype='object', name='gene_name')

In [41]:
print(f"Shape : {expression_matrix.shape}")
expression_matrix.to_csv('expression_matrix.csv')
print("Expression matrix saved!")
print(f"There are {len(labels)} labels, {len(meta_cols)} metadata columns and {len(gene_cols)} genes!")

Shape : (405, 60671)
Expression matrix saved!
There are 2 labels, 9 metadata columns and 60671 genes!


In [ ]:
run_end = time.time()
print(f"This script takes {(run_end-run_start)/60} min to run.")

This script takes 148.16127562522888 to run.
